# How to set the UB matrix directly

**NOTE**: work-in-progress

$UB$ is the 3x3 orientation matrix used to transform coordinates between
reciprocal space directions (of the crystal lattice planes) and the rotational
axes of the diffractometer.

It is possible to set $UB$ directly, as shown next.
Alternatively, $UB$ can be calculated from two non-parallel reflections, using
the method of Busing and Levy (*Acta Cryst* **22** (1967) 457).

## Set UB

For a diffractometer object named `diffractometer`, set $UB$ at `diffractometer.operator.solver.UB`, such as:

```py
diffractometer.operator.solver.UB = [
    [0.5, 6.24, -0.5],
    [-0.5, -0.5, 6.24],
    [-6.24, -0.5, -0.5],
]
```

## Working example

First, create a diffractometer object that uses the `"hkl_soleil"` solver with
the `"hkl"` computation engine.  This solver provides support for many
diffractometer geometries.  This example will use the simulated 4-circle
geometry from the solver's `"E4CV"`.

In [1]:
from hklpy2 import SimulatedE4CV

e4cv = SimulatedE4CV("", name="e4cv")

### Defaults
The diffractometer object starts with a default sample.  The structure is cubic ($a=b=c$, 90 degree corners).

In [2]:
e4cv.sample

Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))

This is the sample the solver will be using.

$U$ is the orientation of the sample's crystal lattice as mounted on the diffractometer
sample holder.  The default is to assume $U=I$, where $I$ is the 3x3 identity
matrix.  

$U$ is provided by the *solver*, in this case `"hkl_soleil"`.  From the
diffractometer object, the solver's default sample $U$ is accessed through a
couple software layers:

In [3]:
e4cv.operator.solver.U

[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]

The default $UB$ (without knowing how the crystal planes are oriented with
respect to the diffractometer) of this cubic crystal is $(2\pi/a) I$ where $I$
is the 3x3 identity matrix and $a$ is the lattice parameter.  Like $U$, this is
provided by the *solver*, in this case `"hkl_soleil"`.

In [4]:
e4cv.operator.solver.UB

[[6.28318530718, -0.0, -0.0],
 [0.0, 6.28318530718, -0.0],
 [0.0, 0.0, 6.28318530718]]

$UB$ is used to transform *(hkl)* to angles (method: `forward()`) and angles to
*(hkl)* (method: `inverse()`).  These transformations are fundamental to
diffractometer operations, so they are provided to the diffractometer object
directly.

Here, we compute the angles for the (111) orientation:

In [5]:
e4cv.forward(1, 1, 1)

SimulatedE4CVRealPos(omega=60.000000002716, chi=35.264389682252, phi=44.999999999738, tth=120.000000005433)

In [6]:
e4cv.operator.solver.mode

'bissector'

Now, convert *(hkl)* from these angles.  Because we have truncated the numerical
precision, we should not expect the precise values of (1.0, 1.0, 1.0).

In [7]:
e4cv.inverse(-60, -35, 45, -120)

SimulatedE4CVPseudoPos(h=-1.003252265133, k=0.993463529784, l=-1.003252265133)

## Set UB to new matrix

The solver's $UB$ matrix can be re-defined by providing a new Python matrix.
Here is a matrix for a cubic sample, oriented previously:

```
[[0.545455316412, -6.239788968842, -0.495930309978],
 [-0.547615630691, -0.543471652084, 6.235639164201],
 [-6.235463558747, -0.498103654451, -0.591011669061]]
```

In [8]:
e4cv.operator.solver.UB = [
    [0.545455316412, -6.239788968842, -0.495930309978],
    [-0.547615630691, -0.543471652084, 6.235639164201],
    [-6.235463558747, -0.498103654451, -0.591011669061]
]
e4cv.operator.solver.UB

[[0.5454561359, -6.239786441844, -0.49593056545],
 [-0.547617660448, -0.543473530637, 6.235639727849],
 [-6.235461050162, -0.498101063505, -0.591009918402]]

In [9]:
e4cv.operator.solver.U

[[0.086812079408, -0.993092986426, -0.078930242493],
 [-0.087156096879, -0.086496568538, 0.992432445262],
 [-0.992404896021, -0.079275872384, -0.094063055512]]

In [10]:
e4cv.inverse(-60, -35, 45, -120)

SimulatedE4CVPseudoPos(h=0.821953663396, k=0.989926119534, l=1.159500990198)

In [11]:
e4cv.forward(1, 1, 1)

SimulatedE4CVRealPos(omega=-59.999921375969, chi=-28.211225311916, phi=40.202361982144, tth=-119.999842751938)

-----

This is for another howto

In [12]:
# # temporary sample
# import math
# e4cv.add_sample("vibranium", 2*math.pi)

In [13]:
r100 = e4cv.add_reflection((1, 0, 0), (-145.451, 5, -5, 69.0966), name="(100)")
r010 = e4cv.add_reflection((0, 1, 0), (-145.451, 5, 85, 69.0966), name="(010)")
r001 = e4cv.add_reflection((0, 0, 1), (-145.451, -85, -95, 69.0966))
e4cv.operator.calcUB(r100, r010)
e4cv.operator.solver.UB


[[0.545455121825, -6.239786696648, -0.495928474853],
 [-0.547615435332, -0.543471167634, 6.235640129207],
 [-6.235461334285, -0.498100449782, -0.591007438011]]

In [14]:
e4cv.forward(1, 0, 0)

SimulatedE4CVRealPos(omega=-150.000014867083, chi=4.999999955784, phi=-4.999297289891, tth=59.999970265834)

In [15]:
e4cv.forward(1, -1, 1)

SimulatedE4CVRealPos(omega=-120.000001897699, chi=-34.931775605417, phi=-44.822655370896, tth=119.999996204601)

In [16]:
r004 = e4cv.add_reflection((1, -1, 1), (-120, -35, -45, 120))
print(f"{e4cv.sample=!r}")
e4cv.operator.refine_lattice()
print(f"{e4cv.sample=!r}")

e4cv.sample=Sample(name='cubic', lattice=Lattice(a=1, b=1, c=1, alpha=90.0, beta=90.0, gamma=90.0))
e4cv.sample=Sample(name='cubic', lattice=Lattice(a=0.8954, b=0.9552, c=0.9222, alpha=86.6391, beta=82.9091, gamma=94.5459))
